In [ ]:
## library
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
import joblib
import json

# load the data

In [ ]:
# syntax import data
from google.colab import files
import io
uploaded = files.upload()

Saving new_data.csv to new_data (1).csv


## data 1

In [ ]:
data1 = pd.read_csv(io.BytesIO(uploaded['new_data (1).csv']))
data1.head()

,Unnamed: 0,User_ID,Gender,Age,Height,Weight,Activities,Total_cal
0,0,14733363,male,68,190.0,94.0,5,3498.66
1,1,14861698,female,20,166.0,60.0,3,2225.49
2,2,11179863,male,69,179.0,79.0,2,2164.39
3,3,16180408,female,34,179.0,71.0,3,2323.45
4,4,17771927,female,27,154.0,58.0,1,1634.52


In [ ]:
data1 = data1.drop(columns=['Unnamed: 0', 'User_ID'])
data1.head()

,Gender,Age,Height,Weight,Activities,Total_cal
0,male,68,190.0,94.0,5,3498.66
1,female,20,166.0,60.0,3,2225.49
2,male,69,179.0,79.0,2,2164.39
3,female,34,179.0,71.0,3,2323.45
4,female,27,154.0,58.0,1,1634.52


In [ ]:
data1.rename(columns={'Total_cal': 'Calories'}, inplace=True)
data1.head()

,Gender,Age,Height,Weight,Activities,Calories
0,male,68,190.0,94.0,5,3498.66
1,female,20,166.0,60.0,3,2225.49
2,male,69,179.0,79.0,2,2164.39
3,female,34,179.0,71.0,3,2323.45
4,female,27,154.0,58.0,1,1634.52


In [ ]:
data1['Gender'] = data1['Gender'].map({'male': 0, 'female': 1})

In [ ]:
data1.shape

(15000, 6)

# model data 1

In [ ]:
def model_data_1():
  numeric_features = ["Age", "Height", "Weight"]
  categorical_features = ["Gender"]
  ordinal_features = ['Activities']
  ct = make_column_transformer((MinMaxScaler(), numeric_features),
                              (OneHotEncoder(handle_unknown='ignore'), categorical_features),
                              (OrdinalEncoder(),ordinal_features))

  # transform the data using transformers
  x = ct.fit_transform(data1)
  y = data1['Calories']

  x_train, x_test = train_test_split(x, train_size=0.80, shuffle=True, random_state=1)
  y_train, y_test = train_test_split(y, train_size=0.80, shuffle=True, random_state=1)

  model = tf.keras.Sequential([
        tf.keras.layers.Dense(11, activation="relu", input_shape = x_train.shape[1:]),
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(50, activation="relu"),
        tf.keras.layers.Dense(50, activation="relu"),
        tf.keras.layers.Dense(85, activation="relu"),
        tf.keras.layers.Dense(100, activation="relu"),
        tf.keras.layers.Dense(1)])
  model.compile(loss=tf.keras.losses.mae,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['mae'])
  model.fit(x_train, y_train, epochs=50, validation_data=[x_test,y_test])

  return model, ct

# save model.h5

In [ ]:
if __name__ == '__main__':
    model, ct = model_data_1()
    # Save the model
    model.save('model_calorie_1.h5')

Epoch 1/50
375/375 [==============================] - 4s 6ms/step - loss: 543.9421 - mae: 543.9421 - val_loss: 53.2670 - val_mae: 53.2670
Epoch 2/50
375/375 [==============================] - 2s 4ms/step - loss: 49.4565 - mae: 49.4565 - val_loss: 43.4695 - val_mae: 43.4695
Epoch 3/50
375/375 [==============================] - 1s 3ms/step - loss: 40.2907 - mae: 40.2907 - val_loss: 45.6606 - val_mae: 45.6606
Epoch 4/50
375/375 [==============================] - 1s 3ms/step - loss: 34.9544 - mae: 34.9544 - val_loss: 29.6748 - val_mae: 29.6748
Epoch 5/50
375/375 [==============================] - 1s 3ms/step - loss: 30.0895 - mae: 30.0895 - val_loss: 26.4265 - val_mae: 26.4265
Epoch 6/50
375/375 [==============================] - 1s 3ms/step - loss: 26.6196 - mae: 26.6196 - val_loss: 23.6345 - val_mae: 23.6345
Epoch 7/50
375/375 [==============================] - 1s 3ms/step - loss: 25.4354 - mae: 25.4354 - val_loss: 20.8925 - val_mae: 20.8925
Epoch 8/50
375/375 [==========================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## tflite to json

In [ ]:
# install
!pip install tensorflowjs

# convert h5 model to json
!tensorflowjs_converter --input_format=keras model_calorie_1.h5 model_calorie_1.json

2024-06-15 15:18:23.596434: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-15 15:18:25.343966: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
